# Matrix Factorization

In this task you are supposed to (manually) implement the matrix factorization variant you learned in the Data Cleaning chapter using the `numpy` library.

In [14]:
import numpy as np

ModuleNotFoundError: No module named 'numpy'

We continue the scenario from the tutorials.

Assume that you have a ginormous database $D$ of three users and three movies and ratings provided by some users to some movies, which we represent as a matrix, where the entry $D_{ij}$ represents the rating user $i$ gave to movie $j$.
Since not all users have rated movies, and the rating ranges from 1 to 5, we encode missing ratings as 0.

In [4]:
# missing values encoded as 0
# 3x3 Matrix
D = [
     [3,1,0],
     [1,0,3],
     [0,3,5],
    ]
D = np.array(D)

N = len(D)
M = len(D)

NameError: name 'np' is not defined

First, randomly initialize the two factors $E$ and $A$ for $f=2$ latent features. For evaluating the correctness of your results from the tutorial, you may *additionally* provide hard-coded inital factors as they have been provided in the tutorial.

In [5]:
# number of latent features
f = 2
# Random matrix E is N*f, so 3x3
E = np.random.rand(3,3)

# Random matrix A is f*M, so 3x3
A = np.random.rand(3,3)

nonzero_indices = np.nonzero(D)
row = nonzero_indices[0][0]
col = nonzero_indices[1][0]
working = D[row, col]
from functools import reduce
e = reduce(lambda x, y: x + y, E[row])
a = 0
for val in D:
    a += val[col]
maybe = e*a
err = pow((working - maybe), 2)

for val in E[row]:
    val = val + 0.001 * 2 * (working - maybe) * a






Implement a function that takes the data matrix $D$, the inital factors $E, A$, the number of epochs (iterations), the learning rate $\eta$, and performs the factorization of $D$. Use a default number of 5000 for the epochs and 0.001 for $\eta$.

Updates to $E$ and $A$ are applied immediately. $\tilde{D}$ is updated after an entry from D was completely dealt with. Update ordered by latent features and E before A.

In [6]:
def cmpt(matrixD, facE, facA, iter = 5000, rate = 0.001):
    
    

Now test your matrix factorization for the parameters sepcified above.

In [7]:
# TODO your code goes here